## Module import

In [1]:
import sqlite3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.graph_objects as go
import ipywidgets as widgets

## Module version

In [2]:
print("plotly version:", plotly.__version__)
print("matplotlib version:", matplotlib.__version__)
print("sqlite3 version:", sqlite3.version)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("ipywidgets version:", widgets.__version__)

plotly version: 4.14.3
matplotlib version: 3.3.4
sqlite3 version: 2.6.0
pandas version: 1.2.2
numpy version: 1.19.2
ipywidgets version: 7.6.3


# Chromosome visualisation

## Functions

Chromosome data recuperation

In [3]:
def stringToList(string):
    return list(string.split(", "))

def getChromInfo(database, chrom_number, info):
    #SQL request
    db_connexion = sqlite3.connect(database)
    
    cursor = db_connexion.cursor()
    
    chrom_sense = cursor.execute("""
    SELECT """+ info +"""
    FROM SGD_features 
    WHERE Chromosome == """+ chrom_number +""" AND Strand == 'W'
    ORDER BY Start_coordinate
    """)
    
    cursor = db_connexion.cursor()

    chrom_antisense = cursor.execute("""
    SELECT """+ info +"""
    FROM SGD_features 
    WHERE Chromosome == """+ chrom_number +""" AND Strand == 'C'
    ORDER BY Stop_coordinate
    """)
    
    List_info = stringToList(info)
    
    #Pandas dataframe formatting
    chrom_sense = chrom_sense.fetchall()
    chrom_sense = pd.DataFrame(chrom_sense, columns=List_info)
    
    chrom_antisense = chrom_antisense.fetchall()
    chrom_antisense = pd.DataFrame(chrom_antisense, columns=List_info)
    
    return chrom_sense, chrom_antisense

Chromosome drawing

In [134]:
def AddChromosome(fig, x1, x2, y1, y2, color, number):
    
    chromosome = go.Scatter(
        x = [x1,x2,x2,x1,x1], 
        y = [y1,y1,y2,y2,y1],
        hoverinfo = 'none',
        marker_symbol = "diamond-tall",
        marker_size = 0.1,
        showlegend = False,
        line_color = color,
        line_width = 0.1,
        fillcolor = color,
        fill = "toself")
    fig.add_trace(chromosome)
    
    chromosome_legend = go.Scatter(
        x = [x1+17000, x2+10000, x2+13000],
        y = [y1+10, y2+15, y2+5],
        #text = ["Chromosome " + number, "Sense", "Antisense"],
        text = [number, "", ""],
        mode = "text",
        showlegend = False,
        hoverinfo = 'none')
    fig.add_trace(chromosome_legend)

Locus drawing

In [135]:
def AddLocus(chrom, fig, y1, y2, parameter, value, color):
    for p,v,c in zip(parameter,value,color) :
        if chrom[p] == v :
            locus = go.Scatter(
                x = [chrom['Start_coordinate'],
                     chrom['Start_coordinate'],
                     chrom['Stop_coordinate'],
                     chrom['Stop_coordinate'],
                     chrom['Start_coordinate']], 
                y = [y1,y2,y2,y1,y1],
                name = chrom['Primary_SGDID'],
                hoveron = 'fills',
                hovertemplate = "<b> ID: %{name}</b>",
                marker_symbol = "diamond-tall",
                marker_size = 0.1,
                legendgroup = v,
                showlegend = False,
                line_color = c,
                line_width = 0.1,
                fillcolor = c,
                fill = "toself")
            fig.add_trace(locus)

Legend drawing

In [136]:
def AddLegend(fig, value, color):
    for v,c in zip(value,color) :
        legend = go.Scatter(
            name = v,
            x = [0], 
            y = [0],
            marker_size = 0.01,
            line_color = c,
            line_width = 0.1,
            fillcolor = c,
            fill = "toself",
            hoverinfo = 'none')
        fig.add_trace(legend)

## Chromosome 1 exemple

In [137]:
#Get data
chrom_1_sense = getChromInfo('../SCERE.db', "1", 
                             """Primary_SGDID, Start_coordinate, Stop_coordinate, Feature_type, Feature_name, Description""")[0]

chrom_1_antisense = getChromInfo('../SCERE.db', "1", 
                                 """Primary_SGDID, Start_coordinate, Stop_coordinate, Feature_type, Feature_name, Description""")[1]

display(chrom_1_sense.head())

#Figure's Layout
layout = go.Layout(plot_bgcolor='white')
fig = go.Figure(layout = layout)
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig.update_xaxes(range=[-10000, 260000])
fig.update_yaxes(range=[-100, 100])

#Add chromosome
AddChromosome(fig, 0, 230218, 0, -20, 'lightgrey', '1')

# Add locus
chrom_1_sense.apply(AddLocus, axis = 1, args =(fig,
                                         0,
                                         -10,
                                         ["Feature_type","Feature_type","Feature_type"],
                                         ["CDS","centromere","telomere"],
                                         ['#B8AC77','black',"black"]))

chrom_1_antisense.apply(AddLocus, axis = 1, args =(fig,
                                            -11,
                                            -20,
                                            ["Feature_type","Feature_type","Feature_type"],
                                            ["CDS","centromere","telomere"],
                                            ['#B8AC77','black',"black"]))

# Add legend
AddLegend(fig,
          ["CDS","centromere","telomere"],
          ['#B8AC77','black',"black"])

fig.show()

,Primary_SGDID,Start_coordinate,Stop_coordinate,Feature_type,Feature_name,Description
0,S000002143,335,649,ORF,YAL069W,Dubious open reading frame; unlikely to encode...
1,S000031098,335,649,CDS,,
2,S000028594,538,792,ORF,YAL068W-A,Dubious open reading frame; unlikely to encode...
3,S000031372,538,792,CDS,,
4,S000028593,2480,2707,ORF,YAL067W-A,Putative protein of unknown function; identifi...


# SCERE whole genome

In [146]:
def Scere_genome(database,chromosomes_number, info, parameter, value, color):
    
    #Figure's Layout
    layout = go.Layout(plot_bgcolor='white')
    fig = go.Figure(layout = layout)
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
    
    #chromosomes length
    db_connexion = sqlite3.connect(database)
    cursor = db_connexion.cursor()
    chromosomes_length = cursor.execute("""
    SELECT length
    FROM chromosome_length
    """)
    chromosomes_length = chromosomes_length.fetchall()
    chromosomes_length = pd.DataFrame(chromosomes_length, columns=["length"])
    
    #chromosomes positions
    Y1 = [0, -40, -80, -120, -160, -200, -240, -280, -320, -360, -400, -440, -480, -520, -560, -600, -640]
    Y2 = [-20, -60, -100, -140, -180, -220, -260, -300, -340, -380, -420, -460, -500, -540, -580, -620, -660]
    
    for c in chromosomes_number :
        #Get data
        chrom_sense = getChromInfo(database, str(c), info)[0]
        
        chrom_antisense = getChromInfo(database, str(c), info)[1]
        
        #Add chromosomes
        AddChromosome(fig, 0, chromosomes_length[c-1:c].values[0,0], Y1[c-1], Y2[c-1], 'lightgrey', str(c))
        
        # Add locus
        chrom_sense.apply(AddLocus, axis = 1, args =(fig,
                                                     Y1[c-1],
                                                     Y2[c-1]+10,
                                                     parameter, 
                                                     value, 
                                                     color))
                                                     
        chrom_antisense.apply(AddLocus, axis = 1, args =(fig,
                                                         Y2[c-1]+9,
                                                         Y2[c-1],
                                                         parameter, 
                                                         value, 
                                                         color))

        
    # Add legend
    AddLegend(fig, value, color)
    
    fig.show()

In [147]:
Scere_genome('../SCERE.db',
             list(range(4,7)),
                """Primary_SGDID, Start_coordinate, Stop_coordinate, Feature_type, Feature_name""",
                ["Feature_type","Feature_type","Feature_type"],
                ["CDS","centromere","telomere"],
                ['#B8AC77','black',"black"])